In [ ]:
## Make some variables with the numbers of rows and columns in the image.
rows, cols = df.header['naxis2'], df.header['naxis1']
print('rows =',rows,'   cols =',cols)

In [ ]:
'''Make a stack of images, a list of headers from those images, and calculate some medians and stds
that will help set autoscaling parameters.'''

# rows, cols = 4924, 7368                    # Number of rows and columns (if not already defined above).
image = np.zeros((len(files), rows,cols))  # 3D numpy array to hold the stack of images.
imedian = np.zeros((len(files)))           # 1D numpy array to hold array medians.
istd = np.zeros((len(files)))              # 1D numpy array to hold array stds.
madstd = np.zeros((len(files)))            # 1D numpy array to hold Median absolute deviations.

headlist = []                              # Empty list to hold the headers.
print('image.shape =', image.shape)

for i in range(len(files)):

# ########################################################################   
#     # Use this code block if you want to work with PipeData objects
#     pd = PipeData(config = config) 
#     pd.load(os.path.join(datapath,files[i]))
#     image[i] = pd.imageget() * 1.0    # Load image data into numpy arrays and convert to float.
#     headlist.append(pd.header)
#     print('')
#     print(i, files[i])
# #########################################################################

#############################################################################
    # Use this code block if you want to use the standard astropy.io package.
    fitsfilename = os.path.join(datapath, files[i])    # Full path to a fitsfile.
    hdulist = fits.open(fitsfilename)                  # Open a fits file as an hdulist object.
    hdu0 = hdulist[0]                                  # Define a fits object as the 0th hdu in the fitsfile.
    image[i] = hdu0.data * 1.0                         # Define image in stack as float of data in the 0th hdu.
    headlist.append(hdu0.header)                       # Append the header of the fits object to the header list.
    print('')
    print(i,files[i])
###############################################################################    
    
    ## Calculate some medians and stds to use in autoscaling that aren't unduly biased by extreme values.
    ## Optionally, print out masked and unmasked values to explore the effects of extreme values on the statistics.
    
    madstd[i] = mad_std(image[i],ignore_nan=True)
    imedian[i] = np.nanmedian(image[i])
    istd[i] = np.nanstd(image[i])
    print(imedian[i], istd[i], madstd[i], np.nanmin(image[i]), np.nanmax(image[i]))

    ## Now make some masks and exclude extreme values
    
    highmask = np.where(image[i] > (imedian[i] + 3 * madstd[i]))
    lowmask = np.where(image[i] < (imedian[i] - 3 * madstd[i]))

    maskedimage = image[i].copy()   # Make a copy to avoid modifying the original image data.
    maskedimage[highmask] = np.nan
    maskedimage[lowmask] = np.nan
    
    imedian_masked = np.nanmedian(maskedimage)
    istd_masked = np.nanstd(maskedimage)
    madstd_masked = mad_std(maskedimage,ignore_nan=True)
    print(imedian_masked, istd_masked, madstd_masked, np.nanmin(maskedimage), np.nanmax(maskedimage))

In [ ]:
#blink images (back and forth)
blink_images(files,image,0,True,blinkset=(0,3),addgrid=False)

In [ ]:
#plot a row and column
whichimage = 0
rowcolplot(image[whichimage],files[whichimage],500,500,autoscale=True,ylims=(0., 65535.),xstep=100,ystep=100)

In [ ]:
'''Take the difference between two images and blink between one and the difference image.'''
A, B = 0,1
nameA = files[A]
nameB = files[B]
imageA = image[A]
imageB = image[B]
difimage = imageA - imageB
difname = 'difference image'

# Compare the 
blink2images(imageA,nameA,difimage,difname,linthresh=100.,linscale=0.3,lims=(-10.,65535.,1.),addgrid=False,xystep=1)

In [ ]:
# make a median image
# Make a median image from the image stack
medianimage = np.nanmedian(image, axis=0)
medim_med = np.nanmedian(medianimage)
medim_std = np.nanstd(medianimage)
medim_madstd = mad_std(medianimage,ignore_nan=True)

## Diagnostics:
## Exclude very large values from the calculation of the std (those greater than 2 times the median).
medimproxy = medianimage.copy()
highmask = np.where(medianimage > medim_med * 2.0) # Criterion for eliminating very large pixel values from the std calculation.
medimproxy[highmask] = np.nan
medimproxymed = np.nanmedian(medimproxy)
medimproxystd = np.nanstd(medimproxy)
medimproxymadstd = mad_std(medimproxy,ignore_nan=True)
print(medim_med, medim_std, medim_madstd)
print(medimproxymed, medimproxystd, medimproxymadstd)

In [ ]:
# # Display difference between images in the stack and the median of the stack.
# figx, figy = 20,20   # Use these to reveal fine detail in images.
figx, figy = 8,8   # Use these to get a quick look and save space.
npic = np.zeros((rows,cols,image.shape[2]))
for i in range(len(files)):
    plt.figure(figsize = (figx,figy))
    npic[i] = image[i] - medianimage
    med = np.nanmedian(npic[i])
    mad = mad_std(npic[i],ignore_nan=True)
    vmx=  med + mad * 3.0
    vmn =  med - mad * 3.0
#     grid()
    plt.title(files[i] + '   Median =' + str(med) + '    mad_std =' + str(mad))
    plt.imshow(npic[i],'gray',interpolation = 'nearest',vmax=vmx,vmin=vmn)
    plt.colorbar(shrink=0.8)

In [ ]:
#noisepic
noisepic = np.nanstd(npic,axis=0)

In [ ]:
'''Filter an image and compare with the original. The idea here is that the filtered image will
substitute a valid value for wildly discrepant pixels and the differenced image will make the bad
pixels stand out.'''

# inimage = image[0][2400:2500,3675:3800].copy()
inimage = noisepic.copy()
imagename = 'noisepic'
fimage = nd.median_filter(inimage,size=3)
fname = 'median filtered image'
gimage = nd.gaussian_filter(fimage,sigma=3)
gname = 'gaussian filter on top of median filter'
difimage = inimage - fimage * 1.0
difname = 'difference image'

# Compare them:
# blink2images(inimage,imagename,difimage,difname,linthresh=100.,linscale=0.3,lims=(-10.,10.,.01),addgrid=True,xystep=10)
blink2images(inimage,imagename,gimage,gname,linthresh=100.,linscale=0.3,lims=(-10.,10.,.01),addgrid=False,xystep=10)

In [ ]:
'''Use scipy.ndimage.labels() to label regions with intensities higher than some value.
Print out for each region the centers of mass, the maximum pixel value, the
sum of the pixel values, the ratio of the sum to the max, and the number of pixels in the region.'''

threshold = 1000

# sub = maskedimage[500:600,25:75]
difmask = np.where(difimage < threshold)
difcopy = difimage.copy()
difcopy[difmask] = 0.
sub = difcopy

difmask100 = np.where(difimage[2400:2500,3675:3800] > threshold)   # This is a test to see what the mask looks like.

# plt.imshow(sub,'gray_r', vmin = 0, vmax = 1)
labels = nd.label(sub)
print( '')

# Find how many areas:
areas = np.max(labels[0])   # The number of areas.
print( areas)
print( '')
# Find out how many pixels are in each labeled area.
pixperarea = np.zeros((areas+1))
print( '')
for i in range(1,areas+1):
    pixperarea[i] = nd.sum(labels[0],labels[0], index = i)/i



print( '')    

lpic = labels[0] * 1.
plt.figure(figsize = (8,8))
plt.imshow(lpic[2400:2500,3675:3800],'gray_r', vmin = 0, vmax = 0.1, interpolation = 'nearest')

plt.figure(figsize = (8,8))
plt.imshow(difimage[2400:2500,3675:3800],'gray_r', interpolation = 'nearest')
plt.colorbar()

plt.figure(figsize = (8,8))
plt.imshow(image[5][2400:2500,3675:3800],'gray_r', interpolation = 'nearest')
plt.colorbar()


# print labels[0]
# test = np.where(maskedimage != 0.)
# print test[0:10]

# for i in range(1,np.max(labels[0])+1):

# Print the whole list
for i in range(1,areas+1):
# for i in range(1,5):       # Diagnostic.
    com = nd.center_of_mass(sub, labels[0], index = i)
#     ext = nd.extrema(sub,labels[0], index = i)      # This one returns the maximum and minimum pixel in the region.
    picmax = nd.maximum(sub,labels[0], index = i)
    picsum = nd.sum(sub,labels[0], index = i)
    ratio = picsum/picmax
#     print i, com[0],com[1], '   ',picsum, '    ', ratio
    print ('{:<5}{:9.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}'.format(i,com[0],com[1],picmax, picsum,ratio, pixperarea[i]))
    
    
# print com2
# print type(labels[0])
# print labels[0].shape
# for i in range(len(labels)):
#     print i, labels[i]
# coms = nd.center_of_mass(maskedimage,labels[1],[1,2])
# print coms

# ## Just print out the areas with number of pixels per region > 2.
# print ''
# n = 0
# for i in range(1,areas+1):
#     com = nd.center_of_mass(sub, labels[0], index = i)
# #     ext = nd.extrema(sub,labels[0], index = i)
#     picmax = nd.maximum(sub,labels[0], index = i)
#     picsum = nd.sum(sub,labels[0], index = i)
#     ratio = picsum/picmax
#     if picmax > 100 and pixperarea[i] > 2.0:
#         n = n+1
#         print ('{:<5}{:9.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}'.format(i,com[0],com[1],picmax, picsum,ratio, pixperarea[i]))
# print ''
# print 'n =',n

In [ ]:
## Print out the mask of a subregion of the image to see what the mask looks like.
print( difmask100)
print( type(difmask100))
difmask100